In [177]:
import pandas as pd
from glob import glob
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
csv_files = glob("data/*.csv")
len(csv_files)

43

#### Look up all the different columns of all kept files

In [17]:
columns = []
for f in tqdm(csv_files):
    df = pd.read_csv(f, encoding="latin-1")
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "")
    columns.append((set(df.columns), f))

100%|██████████| 46/46 [00:24<00:00,  1.90it/s]


In [18]:
unique_columns = []
files = []
for c, f in columns:
    if c not in unique_columns:
        unique_columns.append(c)
        files.append([f])
    else :
        i = unique_columns.index(c)
        files[i].append(f)
column_df = pd.DataFrame(list(zip(unique_columns, files)), columns=["Columns", "Files"])
column_df['num_files'] = column_df.Files.apply(len)
column_df = column_df.sort_values('num_files')

In [19]:
column_df

,Columns,Files,num_files
3,"{yearcode, year, area, monthscode, areacode, e...",[data/Environment_Temperature_change_E_All_Dat...,1
4,"{item, itemcode, yearcode, year, elementcode, ...",[data/Emissions_Agriculture_Energy_E_All_Data_...,1
1,"{item, itemcode, year, elementgroup, elementco...","[data/Environment_Energy_E_All_Data.csv, data/...",4
2,"{item, itemcode, yearcode, year, note, area, a...",[data/Emissions_Agriculture_Burning_crop_resid...,4
0,"{item, itemcode, yearcode, year, area, areacod...",[data/Emissions_Agriculture_Enteric_Fermentati...,36


In [21]:
for i, x in column_df.Columns.iteritems():
    print(x)

{'yearcode', 'year', 'area', 'monthscode', 'areacode', 'element', 'months', 'flag', 'unit', 'elementcode', 'value'}
{'item', 'itemcode', 'yearcode', 'year', 'elementcode', 'element', 'flag', 'unit', 'country', 'value', 'countrycode'}
{'item', 'itemcode', 'year', 'elementgroup', 'elementcode', 'element', 'flag', 'unit', 'country', 'value', 'countrycode'}
{'item', 'itemcode', 'yearcode', 'year', 'note', 'area', 'areacode', 'element', 'flag', 'unit', 'elementcode', 'value'}
{'item', 'itemcode', 'yearcode', 'year', 'area', 'areacode', 'element', 'flag', 'unit', 'elementcode', 'value'}


#### Get mapping of elementcode => (element, unit) and itemcode => item

In [179]:
from mappings import get_all_element_mapping
data_dir = "data/"
element_mapping = get_all_element_mapping(data_dir)

Fount 43 files in data/


In [180]:
from mappings import get_all_item_mapping
item_mapping = get_all_item_mapping(data_dir)

Fount 43 files in data/
Item Code not found in data/Environment_Temperature_change_E_All_Data_(Normalized).csv


#### Print out the description of each csv (**DO NOT RUN**)

In [181]:
from mappings import get_item_mapping, get_element_mapping
from utils import rename_columns
import os


In [182]:
with open("csv_description.md", 'w+') as f:
    for file in csv_files:
        df = rename_columns(pd.read_csv(file, encoding="latin-1"))
        f.write("#### " + os.path.basename(file) + "\n")
        if 'itemcode' in df.columns:
            item_mapping = get_item_mapping(df)
        else:
            item_mapping = pd.DataFrame([])
        element_mapping = get_element_mapping(df)
        f.write("Items : \n" + item_mapping.to_string(index=False) + "\n\n")
        f.write("Elements : \n" + element_mapping.to_string(index=False) + "\n\n")
        f.write("\n\n ------\n\n")